<a href="https://colab.research.google.com/github/cpappas18/Dysarthria-Classifier/blob/main/Getting_the_user_input_and_extracting_features_AI4Good.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Code that retrieves user input in the form of audio and extracts feautures that can be used to check for Parkinsons, ALS and other similar conditions***

In [ ]:
!apt install libasound2-dev portaudio19-dev libportaudio2 libportaudiocpp0 ffmpeg
!pip install PyAudio


Reading package lists... Done
Building dependency tree       
Reading state information... Done
libasound2-dev is already the newest version (1.1.3-5ubuntu0.5).
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 184 kB of archives.
After this operation, 891 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudio2 amd64 19.6.0-1 [64.6 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libportaudiocpp0 amd64 19.6.0-1 [15.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 portaudio19-dev amd64 19.6.0-1 [104 kB]
Fetched 184 kB in 1s (351 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 160772 files and directories currently installed.)
Preparing to 

In [ ]:
!pip install ffmpeg-python

Getting the voice recording

In [ ]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording, press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording..."
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""
#function that records the audio
def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

Record your voice 

In [ ]:
audio, sr = get_audio()

NameError: ignored

Note: If we want to get the vector repr of the audio from a .wav file use this following line of code instead of the get_audio() function

In [ ]:
 import librosa 
 audio, sr = librosa.load('CM13_B1_CW4_M6.wav', sr= None)

In [ ]:
print(sr)

16000


In [ ]:
print(audio)

[-9.1552734e-05 -9.1552734e-05 -9.1552734e-05 ... -5.0048828e-03
 -4.7302246e-03 -4.7912598e-03]


Extracting the relevent features from the audio 

In [ ]:
import librosa
!pip install praat-parselmouth

     |████████████████████████████████| 10.7MB 5.8MB/s 


In [ ]:
!pip install surfboard

from surfboard.sound import Waveform

from surfboard.feature_extraction import extract_features

     |████████████████████████████████| 3.6MB 3.4MB/s 
     |████████████████████████████████| 419kB 27.2MB/s 
     |████████████████████████████████| 81kB 8.5MB/s 
     |████████████████████████████████| 645kB 26.0MB/s 
     |████████████████████████████████| 286kB 27.3MB/s 
     |████████████████████████████████| 20.2MB 1.4MB/s 
  Created wheel for surfboard: filename=surfboard-0.2.0-cp37-none-any.whl size=33231 sha256=ee17b25c70bad478af5101997039a4fe8526056c1a7abbed141ba9a037f7e1c0
  Stored in directory: /root/.cache/pip/wheels/d3/b6/19/72a52f64534dcb26d0c67155ce67e317570cbff6f56cb0f7cf
  Created wheel for pysptk: filename=pysptk-0.1.18-cp37-cp37m-linux_x86_64.whl size=952106 sha256=379788c45cdb189ef0e0f2117162918358d745ae7a5550e25eb7e123b84a7119
  Stored in directory: /root/.cache/pip/wheels/c7/96/d2/a163240019c59504402fab713af259026af81a99dea943404a
Successfully built surfboard pysptk
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is in

Creating a waveform object from our audio in order to get our features 

In [ ]:

sound = Waveform(signal= audio, sample_rate=sr)

print(sound)

The shimmer feature

In [ ]:
shimmers = sound.shimmers()

In [ ]:
print(shimmers)

{'localShimmer': 0.15970584810267952, 'localdbShimmer': 0.8571550117612079, 'apq3Shimmer': 0.08352337616338171, 'apq5Shimmer': 0.10983748750614575, 'apq11Shimmer': 0.27494854441244604}


The jitter feature

In [ ]:
jitters = sound.jitters()

In [ ]:
print(jitters)

{'localJitter': 0.007986416554862446, 'localabsoluteJitter': 6.350231634063103e-05, 'rapJitter': 0.0026129520616990903, 'ppq5Jitter': 0.004211527050334424, 'ddpJitter': 0.007838856185097285}


Another way to get jitters

In [ ]:
!pip install Signal_Analysis
import Signal_Analysis.features.signal

In [ ]:
Signal_Analysis.features.signal.get_Jitter(audio, sr)

<class 'numpy.float64'> 



{'ddp': 0.047114624505939916,
 'local': 0.04813913043478585,
 'local, absolute': 0.0004325131992553777,
 'ppq5': 0.011895652173915158,
 'rap': 0.015704874835313305}

Extracting NHR, HNR, and RPDE

In [ ]:
!pip install Signal_Analysis
import Signal_Analysis.features.signal 

HNR feature extracted (Side Note: this feauture can also be used to detect depression)

In [ ]:
HNR_value = Signal_Analysis.features.signal.get_HNR(audio, sr, time_step=0, min_pitch=75, silence_threshold=0.1, periods_per_window=4.5) 

In [ ]:
print(HNR_value)

3.435519801405292


RPDE

In [ ]:
!pip install pyrpde

In [ ]:
!pip install numba
import pandas as pd
#from surfboard import rpde

In [ ]:
from scipy.io.wavfile import read
import numpy as np
from pyrpde import rpde


/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:365: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [ ]:
#normalizing our array so rpde can take it as a parameter 
normal_audio = audio/(2**16)
print(normal_audio)

[ 2.79396772e-09 -2.79396772e-09 -9.31322575e-10 ... -1.11758709e-08
 -1.39698386e-08 -1.49011612e-08]


In [ ]:
print(normal_audio.dtype)

float64


In [ ]:
#changing the normalized audio from float64 to float32 
final_audio = np.float32(normal_audio)

In [ ]:
print(final_audio)

[ 9.2200935e-08  1.1920929e-07  1.3317913e-07 ... -8.2422048e-08
 -6.2864274e-08 -2.9802322e-08]


In [ ]:
print(final_audio.dtype)

float32


Extraction process using rpde function 

In [ ]:
#Note: histogram is a 1-dimensional array corresponding to the histogram of the return distances
rpde_value, histogram = rpde(final_audio, tau=30, dim=4, epsilon=0.01, tmax=1500)

RPDE Feature Extracted

In [ ]:
print(rpde_value)

0.0


Getting DFA Feature 

In [ ]:
import surfboard
#1024 is the default window_length
dfa_feature = surfboard.dfa.get_dfa(audio, [1024])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RankWarning: Polyfit may be poorly conditioned
  This is separate from the ipykernel package so we can avoid doing imports until


DFA Feature Extracted

In [ ]:
print(audio)

[-9.1552734e-05 -9.1552734e-05 -9.1552734e-05 ... -5.0048828e-03
 -4.7302246e-03 -4.7912598e-03]


In [ ]:
print(dfa_feature)

0.1455661793431911


Extra Feature: Fundamental Mean Frequency

In [ ]:
!pip install Signal_Analysis
import Signal_Analysis.features.signal 

F_0 = Signal_Analysis.features.signal.get_F_0(audio, sr, time_step=0, min_pitch=75) 

<class 'numpy.float64'> 



In [ ]:
print(F_0)

[113.57142857142856]


Note: Can also use librosa import pitch to get Fundamental mean frequency

VISUALIZATION OF SPEECH FEATURES

Speech Amplitudes of Healthy Control vs ALS Patients

In [ ]:
import parselmouth
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#ALS
sns.set() # Using seaborn's default style 

# Plot nice figures using Python's "standard" matplotlib library
snd_als = parselmouth.Sound("SPA2019_Pathology_008.wav") 
plt.figure()
plt.plot(snd_als.xs(), snd_als.values.T)
plt.xlim([snd_als.xmin, snd_als.xmax])
plt.xlabel("time [s]")
plt.ylabel("amplitude")
plt.show() # or plt.savefig("sound.png"), or plt.savefig("sound.pdf")

#Healthy Control
sns.set() # Using seaborn's default style 

# Plotting Spectogram for HC
snd_HC = parselmouth.Sound("SPA2019_Control_002.wav") 
plt.figure()
plt.plot(snd_HC.xs(), snd_HC.values.T)
plt.xlim([snd_HC.xmin, snd_HC.xmax])
plt.xlabel("time [s]")
plt.ylabel("amplitude")
plt.show() # or plt.savefig("sound.png"), or plt.savefig("sound.pdf")

PraatError: ignored

Spectograms of Healthy vs ALS Patient


In [ ]:
#ALS
def draw_spectrogram_als(spectrogram_als, dynamic_range=70):
    X, Y = spectrogram.x_grid(), spectrogram.y_grid()
    sg_db = 10 * np.log10(spectrogram.values)
    plt.pcolormesh(X, Y, sg_db, vmin=sg_db.max() - dynamic_range, cmap='afmhot')
    plt.ylim([spectrogram.ymin, spectrogram.ymax])
    plt.xlabel("time [s]")
    plt.ylabel("frequency [Hz]")

def draw_intensity_als(intensity_als):
    plt.plot(intensity.xs(), intensity.values.T, linewidth=3, color='w')
    plt.plot(intensity.xs(), intensity.values.T, linewidth=1)
    plt.grid(False)
    plt.ylim(0)
    plt.ylabel("intensity [dB]")

#HC
def draw_spectrogram_hc(spectrogram_HC, dynamic_range=70):
    X, Y = spectrogram.x_grid(), spectrogram.y_grid()
    sg_db = 10 * np.log10(spectrogram.values)
    plt.pcolormesh(X, Y, sg_db, vmin=sg_db.max() - dynamic_range, cmap='afmhot')
    plt.ylim([spectrogram.ymin, spectrogram.ymax])
    plt.xlabel("time [s]")
    plt.ylabel("frequency [Hz]")

def draw_intensity_hc(intensity_HC):
    plt.plot(intensity.xs(), intensity.values.T, linewidth=3, color='w')
    plt.plot(intensity.xs(), intensity.values.T, linewidth=1)
    plt.grid(False)
    plt.ylim(0)
    plt.ylabel("intensity [dB]")
    
#ALS
intensity_als = snd_als.to_intensity()
spectrogram_als = snd_als.to_spectrogram()
plt.figure()
draw_spectrogram_als(spectrogram_als)
plt.twinx()
draw_intensity_als(intensity_als)
plt.xlim([snd_als.xmin, snd_als.xmax])
plt.show() 

#HC
intensity_HC = snd_HC.to_intensity()
spectrogram_HC = snd_HC.to_spectrogram()
plt.figure()
draw_spectrogram_hc(spectrogram_HC)
plt.twinx()
draw_intensity_hc(intensity_HC)
plt.xlim([snd_HC.xmin, snd_HC.xmax])
plt.show() 

Pitch in HC and PD Subjects

In [ ]:
#ALS
def draw_pitch(pitch_als):
    # Extract selected pitch contour, and
    # replace unvoiced samples by NaN to not plot
    pitch_values = pitch.selected_array['frequency']
    pitch_values[pitch_values==0] = np.nan
    plt.plot(pitch.xs(), pitch_values, 'o', markersize=5, color='w')
    plt.plot(pitch.xs(), pitch_values, 'o', markersize=2)
    plt.grid(False)
    plt.ylim(0, pitch.ceiling)
    plt.ylabel("fundamental frequency [Hz]")

pitch_als = snd_als.to_pitch()
# If desired, pre-emphasize the sound fragment before calculating the spectrogram
pre_emphasized_snd = snd_als.copy()
pre_emphasized_snd.pre_emphasize()
spectrogram = pre_emphasized_snd.to_spectrogram(window_length=0.03, maximum_frequency=8000)
plt.figure()
draw_spectrogram(spectrogram)
plt.twinx()
draw_pitch(pitch_als)
plt.xlim([snd_als.xmin, snd_als.xmax])
plt.show() # or plt.savefig("spectrogram_0.03.pdf")

#HC
def draw_pitch(pitch_HC):
    # Extract selected pitch contour, and
    # replace unvoiced samples by NaN to not plot
    pitch_values = pitch.selected_array['frequency']
    pitch_values[pitch_values==0] = np.nan
    plt.plot(pitch.xs(), pitch_values, 'o', markersize=5, color='w')
    plt.plot(pitch.xs(), pitch_values, 'o', markersize=2)
    plt.grid(False)
    plt.ylim(0, pitch.ceiling)
    plt.ylabel("fundamental frequency [Hz]")

pitch_HC = snd_HC.to_pitch()
# If desired, pre-emphasize the sound fragment before calculating the spectrogram
pre_emphasized_snd_hc = snd_HC.copy()
pre_emphasized_snd_hc.pre_emphasize()
spectrogram = pre_emphasized_snd_hc.to_spectrogram(window_length=0.03, maximum_frequency=8000)
plt.figure()
draw_spectrogram(spectrogram)
plt.twinx()
draw_pitch(pitch_HC)
plt.xlim([snd_HC.xmin, snd_HC.xmax])
plt.show() # or plt.savefig("spectrogram_0.03.pdf")